<a href="https://cognitiveclass.ai"><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork/images/IDSN-logo.png" width="400"> </a>

# Convolutional Neural Networks with Keras

Estimated time needed **30** mins


In this lab, we will learn how to use the Keras library to build convolutional neural networks. We will also use the popular MNIST dataset and we will compare our results to using a conventional neural network.


## Objectives for this Notebook    
* How to use the Keras library to build convolutional neural networks
* Convolutional neural network with one set of convolutional and pooling layers
* Convolutional neural network with two sets of convolutional and pooling layers



## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
      
1. <a href="#Import-Keras-and-Packages">Import Keras and Packages</a>   
2. <a href="#Convolutional-Neural-Network-with-One-Set-of-Convolutional-and-Pooling-Layers">Convolutional Neural Network with One Set of Convolutional and Pooling Layers</a>  
3. <a href="#Convolutional-Neural-Network-with-Two-Sets-of-Convolutional-and-Pooling-Layers">Convolutional Neural Network with Two Sets of Convolutional and Pooling Layers</a>  

</font>
</div>


### Install the necessary libraries


Let's start by installing the keras libraries and the packages that we would need to build a neural network.


In [1]:
# All Libraries required for this lab are listed below. The libraries need to be installed on Skills Network Labs. 
# If you run this notebook on a different environment, e.g. your desktop, you may want to install these.
!pip install numpy==2.0.2
!pip install pandas==2.2.2
!pip install tensorflow_cpu==2.18.0
!pip install matplotlib==3.9.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 133.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 MB 42.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 48.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5

#### Suppress the tensorflow warning messages
We use the following code to  suppress the warning messages due to use of CPU architechture for tensoflow.
You may want to **comment out** these lines if you are using the GPU architechture


In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Import Keras and Packages


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from keras.utils import to_categorical

When working with convolutional neural networks in particular, we will need additional packages.


In [4]:
from keras.layers import Conv2D # to add convolutional layers
from keras.layers import MaxPooling2D # to add pooling layers
from keras.layers import Flatten # to flatten data for fully connected layers

## Convolutional Neural Network with One Set of Convolutional and Pooling Layers


In [5]:
# import data
from keras.datasets import mnist

# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Let's normalize the pixel values to be between 0 and 1


In [6]:
X_train = X_train / 255 # normalize training data
X_test = X_test / 255 # normalize test data

Next, let's convert the target variable into binary categories


In [7]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1] # number of categories

Next, let's define a function that creates our model. Let's start with one set of convolutional and pooling layers.


In [8]:
def convolutional_model():
    
    # create model
    model = Sequential()
    model.add(Input(shape=(28, 28, 1)))
    model.add(Conv2D(16, (5, 5), strides=(1, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    return model

Finally, let's call the function to create the model, and then let's train it and evaluate it.


In [9]:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/10
300/300 - 37s - 124ms/step - accuracy: 0.9255 - loss: 0.2754 - val_accuracy: 0.9744 - val_loss: 0.0862
Epoch 2/10
300/300 - 34s - 112ms/step - accuracy: 0.9776 - loss: 0.0768 - val_accuracy: 0.9820 - val_loss: 0.0592
Epoch 3/10
300/300 - 34s - 115ms/step - accuracy: 0.9844 - loss: 0.0534 - val_accuracy: 0.9841 - val_loss: 0.0504
Epoch 4/10
300/300 - 33s - 111ms/step - accuracy: 0.9871 - loss: 0.0425 - val_accuracy: 0.9865 - val_loss: 0.0417
Epoch 5/10
300/300 - 34s - 113ms/step - accuracy: 0.9893 - loss: 0.0345 - val_accuracy: 0.9868 - val_loss: 0.0395
Epoch 6/10
300/300 - 35s - 116ms/step - accuracy: 0.9915 - loss: 0.0274 - val_accuracy: 0.9884 - val_loss: 0.0381
Epoch 7/10
300/300 - 34s - 112ms/step - accuracy: 0.9932 - loss: 0.0223 - val_accuracy: 0.9894 - val_loss: 0.0329
Epoch 8/10
300/300 - 34s - 112ms/step - accuracy: 0.9942 - loss: 0.0193 - val_accuracy: 0.9882 - val_loss: 0.0360
Epoch 9/10
300/300 - 33s - 111ms/step - accuracy: 0.9949 - loss: 0.0163 - val_accuracy: 

------------------------------------------


## Convolutional Neural Network with Two Sets of Convolutional and Pooling Layers


Let's redefine our convolutional model so that it has two convolutional and pooling layers instead of just one layer of each.


In [10]:
def convolutional_model():
    
    # create model
    model = Sequential()
    model.add(Input(shape=(28, 28, 1)))
    model.add(Conv2D(16, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(8, (2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    return model

Now, let's call the function to create our new convolutional neural network, and then let's train it and evaluate it.


In [11]:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/10
300/300 - 39s - 129ms/step - accuracy: 0.8541 - loss: 0.4929 - val_accuracy: 0.9542 - val_loss: 0.1592
Epoch 2/10
300/300 - 37s - 123ms/step - accuracy: 0.9606 - loss: 0.1292 - val_accuracy: 0.9711 - val_loss: 0.0910
Epoch 3/10
300/300 - 37s - 124ms/step - accuracy: 0.9722 - loss: 0.0893 - val_accuracy: 0.9795 - val_loss: 0.0645
Epoch 4/10
300/300 - 36s - 120ms/step - accuracy: 0.9787 - loss: 0.0702 - val_accuracy: 0.9798 - val_loss: 0.0641
Epoch 5/10
300/300 - 36s - 120ms/step - accuracy: 0.9815 - loss: 0.0593 - val_accuracy: 0.9816 - val_loss: 0.0573
Epoch 6/10
300/300 - 44s - 147ms/step - accuracy: 0.9840 - loss: 0.0528 - val_accuracy: 0.9851 - val_loss: 0.0470
Epoch 7/10
300/300 - 51s - 171ms/step - accuracy: 0.9860 - loss: 0.0459 - val_accuracy: 0.9865 - val_loss: 0.0413
Epoch 8/10
300/300 - 52s - 174ms/step - accuracy: 0.9866 - loss: 0.0425 - val_accuracy: 0.9871 - val_loss: 0.0400
Epoch 9/10
300/300 - 51s - 170ms/step - accuracy: 0.9887 - loss: 0.0370 - val_accuracy: 

<h3>Practice Exercise 1</h3>


Let's see how batch size affects the time required and accuracy of the model training. 
For this, you can try to change batch_size to 1024 and check it's effect on accuracy


In [14]:
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/10
59/59 - 36s - 604ms/step - accuracy: 0.6718 - loss: 1.2079 - val_accuracy: 0.8978 - val_loss: 0.3563
Epoch 2/10
59/59 - 42s - 704ms/step - accuracy: 0.9145 - loss: 0.2895 - val_accuracy: 0.9377 - val_loss: 0.2068
Epoch 3/10
59/59 - 42s - 709ms/step - accuracy: 0.9453 - loss: 0.1866 - val_accuracy: 0.9586 - val_loss: 0.1422
Epoch 4/10
59/59 - 41s - 695ms/step - accuracy: 0.9582 - loss: 0.1413 - val_accuracy: 0.9657 - val_loss: 0.1173
Epoch 5/10
59/59 - 42s - 710ms/step - accuracy: 0.9644 - loss: 0.1177 - val_accuracy: 0.9718 - val_loss: 0.0964
Epoch 6/10
59/59 - 41s - 688ms/step - accuracy: 0.9696 - loss: 0.1008 - val_accuracy: 0.9755 - val_loss: 0.0832
Epoch 7/10
59/59 - 42s - 709ms/step - accuracy: 0.9727 - loss: 0.0906 - val_accuracy: 0.9759 - val_loss: 0.0770
Epoch 8/10
59/59 - 42s - 709ms/step - accuracy: 0.9736 - loss: 0.0849 - val_accuracy: 0.9785 - val_loss: 0.0676
Epoch 9/10
59/59 - 41s - 694ms/step - accuracy: 0.9771 - loss: 0.0760 - val_accuracy: 0.9794 - val_loss:

Double-click <b>here</b> for the solution.

<!-- Your answer is below:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))
-->


<h3>Practice Exercise 2</h3>


Now, let's see how number of epochs  affect the time required and accuracy of the model training. 
For this, you can keep the batch_size=1024 and epochs=25 and check it's effect on accuracy


In [ ]:
# Write your answer here

model = convolutional_model()

model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 25, batch_size = 1024, verbose = 2)

scores = model.evaluate(X_test, y_test, verbose = 0) 
print("Accuracy: {} \n Error: {}",format(scores[1], 100 - scores[1]* 100))

Double-click <b>here</b> for the solution.

<!-- Your answer is below:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))


    -->


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!


<!--
## Change Log

|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2024-11-20  | 3.0  | Aman  |  Updated the library versions to current |
| 2020-09-21  | 2.0  | Srishti  |  Migrated Lab to Markdown and added to course repo in GitLab |



<hr>

## <h3 align="center"> © IBM Corporation. All rights reserved. <h3/>


## <h3 align="center"> &#169; IBM Corporation. All rights reserved. <h3/>

